In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
import shap
from catboost import *
from sklearn.preprocessing import label_binarize
from catboost import *
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

from sklearn.metrics import r2_score
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('database_filled_TOF_MCL.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                       'Ionization Potential',#0
                      'Electronegativity',#1
                      'Number of d electrons',#2
                      'Graphene/Carbon Nanosheets or other 2D Structures',#3
                      'Carbon Nanofiber/Nanotubes',#4
                      'Biomass or other Organic Derived',#5  
                      'Main Transition Metal Content (wt. %)',#6
                      'Nitrogen Cotent (wt. %)',#7
                      'Metal-N Coordination Number (XAS)',#8    
                      'Pyridinic N Ratio',#9
                      'Pyrrolic N Ratio',#10
                      'Raman ID/IG Ratio',#11
                      'BET Surface Area (m2/g)',#12
                      'Pyrolysis Temperature (°C)',#13
                      'Pyrolysis Time (h)',#14
                      'Rising Rate (°C min-1)',#15
                      'Flow Cell/H-type Cell',#16
                      'Electrolyte Concentration (M)',#17
                      'Catalyst Loading (mg cm-2)',#18
                      'Carbon Paper/Glassy Carbon',#19
                      'Electrolyte pH',#20
                      'TOF@Maximum FE(h-1) (log)'#21
                        ]]
###########defining a wrapper function for later call from each machine learning algorithms##########
raw_param=raw_data.iloc[:,0:21]
raw_power=raw_data.iloc[:,21]
# raw_input_global=raw_data.iloc[:,0:32]
# raw_output_global=raw_data.iloc[:,32]
X=raw_param.values.astype(np.float32)
y=raw_power.values.astype(np.float32)
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})


In [ ]:
def gridsearch(model,param,algorithm_name):
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test.values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    
    result_train = best_model.predict(X_train)
    x_prediction_07_train=result_train
    y_real_07_train=y_train.values
    x_prediction_07_series_train=pd.Series(x_prediction_07_train)
    y_real_07_series_train=pd.Series(y_real_07_train)
    
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    
    corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
    error_val_train= compute_mae_mse_rmse(x_prediction_07_train,y_real_07_train)
    
    print(algorithm_name)
    print(best_model.feature_importances_)
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val,'TEST R2',error_val[3],'TEST CORR',corr_ann)
    print(error_val_train,'TRAIN R2',error_val_train[3],'TRAIN CORR',corr_ann_train)
    x_y_x=np.arange(0,5,0.1)
    x_y_y=np.arange(0,5,0.1)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name+' Test Set',alpha=0.75)
    ax.scatter(x_prediction_07_train,y_real_07_train,color='blue',label=algorithm_name+' Training Set',alpha=0.25,marker="^")
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Log(TOF(h-1))@_Maximum_Faradaic_Efficiency")
    plt.ylabel(u"Real_Log(TOF(h-1))@_Maximum_Faradaic_Efficiency")
    print('finished')
    return best_model
def shap_plot_interaction(best_model,algorithm_name,interacted_features):
    print(algorithm_name)
    SHAP_INPUT=raw_param
    SHAP_OUTPUT=raw_power
    print('train finished')
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)

    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)        
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT) 
        print(shap_values)
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
    elif algorithm_name=='ANN':
        SHAP_INPUT=standardized_data.iloc[:,0:21]
        SHAP_OUTPUT=raw_data.iloc[:,21]
        X_SHAP=SHAP_INPUT.values.astype(np.float32)
        y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
        explainer = shap.DeepExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP) 
        print(shap_values)
        shap.dependence_plot(interacted_features[0], shap_values[0], SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values[0], SHAP_INPUT,interaction_index= interacted_features[0])
    else:
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
from pdpbox import pdp
def plot_pdp_interact_ANN(model, df, f_list, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    inter1 = pdp.pdp_interact(model, df, model_features=df.columns.tolist(), features=f_list,num_grid_points=[20,20])
    # plot it
    settings = {
            'contour_color':  'white',
            'font_family': 'Arial',
            # matplotlib color map for interact plot
            'cmap': 'viridis',
            # fill alpha for interact plot
            'inter_fill_alpha': 0.8,
            # fontsize for interact plot text
            'inter_fontsize': 7,
        }
    pdp.pdp_interact_plot(
    pdp_interact_out=inter1, feature_names=f_list, plot_type='contour',figsize=(10,10),x_quantile=True, plot_pdp=True,plot_params=settings)
from sklearn.inspection import plot_partial_dependence
from sklearn.utils import validation
def pdp_plot_2d(best_model,f_list):
    print('start')
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_list], X=raw_param, percentiles=(0, 1),grid_resolution=100)
def pdp_plot_2d_XG_CAT(best_model,f_list):
    print('start')
    best_model.dummy_ = "dummy"
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_list], X=raw_param, percentiles=(0, 1),grid_resolution=100)
def pdp_plot_2d_ANN(model,f_list):
    print('start')
    model.dummy_ = "dummy"
    print(type(model))
    validation.check_is_fitted(estimator=model)
    my_plots =plot_partial_dependence(model, features=[f_listt], X=raw_input, percentiles=(0, 1))

In [ ]:
seed=9
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.1,random_state=seed)

In [ ]:
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMRegressor=lightgbm.LGBMRegressor(random_state=1,verbose=-1)
param_light = {
 'boosting_type':['gbdt'],
 'learning_rate':[0.1],
 'n_estimators':[200],
    'max_depth':[7],
 'subsample':[0.4],
 'reg_alpha':[0.001],
 'reg_lambda':[0.001]
}
LGBM=gridsearch(model_LightGBMRegressor,param_light,'LightGBM')

In [ ]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBRegressor=xgboost.XGBRegressor(objective='reg:squarederror',random_state=1,verbose=0)
param_xg = {
 'booster':['gbtree'],
 'learning_rate':[0.025],
 'n_estimators':[400],
 'max_depth':[9],
 'subsample':[0.4],
 'reg_alpha':[0.001],
 'reg_lambda':[0.001]
}
XG=gridsearch(model_XGBRegressor,param_xg,'XGBoost')

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
 'learning_rate':[0.075],
 'n_estimators':[400],
 'max_depth':[5],
 'subsample':[0.4],
    'reg_lambda':[1e-05]
}
CAT=gridsearch(model_CatRegressor,param_cat,'CatBoost')

In [ ]:
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)

###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:21]
raw_output=standardized_data.iloc[:,21]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
raw_input_global=raw_data.iloc[:,0:21]
raw_output_global=raw_data.iloc[:,21]
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
def intergate(y_pred):
    length=y_pred.shape[0]
    print(length)
    for i in range (0,length):
        if y_pred[i][0]>=0.5:
            y_pred[i][0]=1
        else:
            y_pred[i][0]=0
    return y_pred
early_stopping=keras.callbacks.EarlyStopping(
 monitor="val_loss", 
 patience=20, 
 verbose=0, 
 mode="auto"
)
for batch_size_number in [24]:
    for reg in [0.0001]:
        for dropout_rate in [0]:
            for neurons1 in [300]:
                for epochs_number in [550]:
                    for learning_rate_search in [0.02]:
                        for activation1 in ['relu']:
                            regularizer=keras.regularizers.l2(reg)
                            ###########keras ANN model construction##########
                            model = Sequential() 
                            model.add(Dense(neurons1, input_dim=21, kernel_initializer='random_normal',
                                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate))
                            model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                            bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                            model.add(Dropout(dropout_rate))
                            model.add(Dense(1, input_dim=neurons1, activation='linear'))
                            adam=optimizers.Adam(lr=learning_rate_search)
                            model.compile(loss='mse', optimizer=adam)
                            print('training...')
                            model.fit(X_train, y_train,verbose=0, batch_size=batch_size_number,epochs=epochs_number,validation_split=0.2,callbacks=[early_stopping])
                            result=model.predict(X_test)
                            result_train=model.predict(X_train)
#                             ###########get RMSE and R2 on the test set##########
#                             x_prediction_07=result*np.std(raw_output_global,axis=0)+np.mean(raw_output_global,axis=0)
#                             y_real_07=np.std(raw_output_global,axis=0)*y_test+np.mean(raw_output_global,axis=0)
#                             x_prediction_07_series=pd.Series(x_prediction_07[:,0])
#                             y_real_07_series=pd.Series(y_real_07)
#                             #training set
#                             x_prediction_07_train=result_train*np.std(raw_output_global,axis=0)+np.mean(raw_output_global,axis=0)
#                             y_real_07_train=np.std(raw_output_global,axis=0)*y_train+np.mean(raw_output_global,axis=0)
#                             x_prediction_07_series_train=pd.Series(x_prediction_07_train[:,0])
#                             y_real_07_series_train=pd.Series(y_real_07_train)
#                             ###########evaluating the regression quality##########
#                             corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
#                             error_val= compute_mae_mse_rmse(x_prediction_07[:,0],y_real_07)
#                             corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
#                             error_val_train= compute_mae_mse_rmse(x_prediction_07_train[:,0],y_real_07_train)
#                             print('TEST SET scatter corr',corr_ann,'scatter error',error_val,'TEST R2',error_val[3])
#                             print('TRAINING SET scatter corr',corr_ann_train,'scatter error',error_val_train,'R2',error_val_train[3])
#                             print(neurons1,epochs_number,learning_rate_search,dropout_rate,batch_size_number,reg,activation1)                      
#                             x_y_x=np.arange(0.2,1.1,0.1)
#                             x_y_y=np.arange(0.2,1.1,0.1)
#                             fig = plt.figure()
#                             ax = fig.add_subplot(111)
#                             ax.scatter(x_prediction_07[:,0],y_real_07,color='red',label='Artificial Neural Network Test Set',alpha=0.75)
#                             ax.scatter(x_prediction_07_train[:,0],y_real_07_train,color='blue',label='Artificial Neural Network Training Set',alpha=0.25,marker="^")
#                             ax.plot(x_y_x,x_y_y)
#                             plt.legend()
#                             plt.xlabel(u"Predicted_Overpotential_@_Maximum_Faradic_Efficiency V (vs. RHE)")
#                             plt.ylabel(u"Real_Overpotential_@_Maximum_Faradic_Efficiency V (vs. RHE)")
#                             plt.savefig('newest 0.678 %s %s %s %s %s %s %s.png' %(neurons1,epochs_number,learning_rate_search,dropout_rate,batch_size_number,reg,activation1))
#                             plt.show()
ANN=model

In [ ]:
shap_plot_interaction(LGBM,algorithm_name="LightGBM",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d_XG_CAT(LGBM,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
shap_plot_interaction(XG,algorithm_name="XGBoost",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d_XG_CAT(XG,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
shap_plot_interaction(CAT,algorithm_name="CatBoost",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
pdp_plot_2d_XG_CAT(CAT,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
shap_plot_interaction(ANN,algorithm_name="ANN",interacted_features=['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])

In [ ]:
plot_pdp_interact_ANN(ANN,raw_input,['Flow Cell/H-type Cell','Catalyst Loading (mg cm-2)'])